# **Cài đặt và gọi các thư viện**

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import urllib
!pip install pyodbc #Cài đặt thư viện pyodbc
!pip install sqlalchemy #Cài đặt thư viện sqlalchemy
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine, text, Table, Column, String, Float, Integer, DateTime, MetaData

# **Tiến hành ETL to database**

In [ ]:
class DatabaseManager:
    def __init__(self, server, database, username, password,df,table_name):
        """
        Khởi tạo đối tượng DatabaseManager.

        Parameters:
            server (str): Tên server SQL Server.
            database (str): Tên database.
            username (str): Tên tài khoản đăng nhập.
            password (str): Mật khẩu tài khoản.
        """
        self.server = server
        self.database = database
        self.username = username
        self.password = password
        self.df = df
        self.table_name = table_name
        self.engine = None

    def connect_engine(self):
        """
        Tạo kết nối engine cho SQLAlchemy.
        """
        try:
            params = urllib.parse.quote_plus(
                f"DRIVER={{ODBC Driver 17 for SQL Server}};"
                f"SERVER={self.server};"
                f"DATABASE={self.database};"
                f"UID={self.username};"
                f"PWD={self.password};"
                f"Trusted_Connection=no;"
            )
            self.engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")
            print("Kết nối SQLAlchemy engine thành công.")
        except Exception as e:
            print(f"Không thể kết nối SQLAlchemy engine. Lỗi: {e}")

    def create_database_if_not_exists(self):
        """
        Kiểm tra và tạo database nếu chưa tồn tại.
        """
        conn_str = (
            f"DRIVER={{ODBC Driver 17 for SQL Server}};"
            f"SERVER={self.server};"
            f"UID={self.username};"
            f"PWD={self.password};"
            f"Trusted_Connection=no;"
        )
        try:
            with pyodbc.connect(conn_str, autocommit=True) as conn:
                cursor = conn.cursor()
                cursor.execute(f"IF NOT EXISTS(SELECT * FROM sys.databases WHERE name='{self.database}') CREATE DATABASE {self.database}")
                print(f"Đã kiểm tra và tạo database '{self.database}' nếu chưa tồn tại.")
        except Exception as e:
            print(f"Lỗi khi tạo database: {e}")
    def prepare_unicode_data(self):
        """Chuyển đổi dữ liệu chuỗi thành Unicode (N'') để hỗ trợ tiếng Việt."""
        def to_unicode(value):
            if isinstance(value, str):  # Chỉ áp dụng với chuỗi
                return "N'" + value.replace("'", "''") + "'"
            return value
        self.df = self.df.applymap(to_unicode)
    def load_data(self):
        # Tên bảng
        table_name = self.table_name

        # Xác định độ dài tối đa cho mỗi cột (dựa trên độ dài tối đa của dữ liệu trong mỗi cột)
        dtype_dict = {}
        for col in self.df.columns:
            # Tính độ dài tối đa của từng cột
            max_length = self.df[col].astype(str).map(len).max()  # Tính chiều dài tối đa của mỗi cột
            dtype_dict[col] = sqlalchemy.types.NVARCHAR(length=max_length)

        # Ghi DataFrame vào bảng trong cơ sở dữ liệu
        with self.engine.connect() as connection:
            # Tạo bảng và chèn dữ liệu
            self.df.to_sql(
                table_name,
                con=connection,
                if_exists='replace',  # Xóa bảng cũ nếu tồn tại
                index=False,
                dtype=dtype_dict  # Định nghĩa kiểu dữ liệu cho mỗi cột với chiều dài tối đa
            )
            print(f"Dữ liệu đã được lưu thành công vào bảng '{table_name}'.")
def extract_data():
    df= pd.read_excel('vietnamwork.xlsx')
    return df
def check_data(df):
    print(df.head())
    print(df.info())
    print(df.duplicated().sum())
    print(df.isnull().sum())
def transform_data(df):
    df=df.drop('index_', axis=1)
    df = df[df['Địa điểm'].notna()].reset_index(drop=True)
    df = df.fillna(value="Không hiển thị")
    df["Ngành nghề"] = df["Ngành nghề"].str.replace(">", ",").str.rstrip()
    df["Từ khóa"] = df["Từ khóa"].str.replace("\n", ",").str.rstrip()
    df['tag'] = df[['Kỹ năng', 'Ngành nghề', 'Từ khóa', 'Công ty', 'Lĩnh vực']].apply(
    lambda row: ', '.join(dict.fromkeys([item.strip() for item in ','.join(row.dropna()).split(',') if item.strip()])),axis=1)
    df=df.drop('Từ khóa', axis=1)
    return df
df=extract_data()
check_data(df)
data=transform_data(df)
if __name__ == "__main__":
    server = r'DESKTOP-Q6B5CSD\NAMNH' # Thay thế lại server của bạn
    database = 'recommend_job' #Tên Database của bạn
    username = 'sa' #Tài khoản của bạn
    password = 'Nam@15092003' #Mật khẩu của bạn
    db_manager = DatabaseManager(server, database, username, password,data,'job')
    db_manager.create_database_if_not_exists()
    db_manager.connect_engine()
    db_manager.load_data()

